In [18]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [19]:
# img import 및 그레이스케일 화
img = cv2.imread('img/20230516182409_00001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img = cv2.GaussianBlur(img, (5,5), 0)

In [20]:
# 색상 반전
# invert = img.copy()
# invert = 255 - invert
# invert.size

In [21]:
# 임계값 지정
ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV)

cv2.imshow("2", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
# 가장 넓이가 큰 경계값 찾기 ( 중앙의 결과값 테이블 )
images, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
images = cv2.cvtColor(images, cv2.COLOR_GRAY2BGR)
contour_array = sorted(contours, key=cv2.contourArea, reverse=True)

table_data = contour_array[0]
x, y, w, h = cv2.boundingRect(table_data)

print(x, y, w, h)

256 623 1258 1039


In [23]:
# 칸마다 마진을 두어 탐색 진행

col_count = 8
row_count = 20

margin = [4, 2]

cell_width = round(w / col_count) - margin[0]
cell_height = round(h / row_count) - margin[1]

# cv2.putText(images, "BIG AREA", (x, y), cv2.FONT_ITALIC, 10, (255, 255, 0), 2)
# cv2.rectangle(images, (x, y), (x + w, y + h), (255, 255, 0), 3)

In [24]:
coord = [x, y]
img_arr = []
for y_ in range(row_count):
    temp = []
    for x_ in range(col_count):
        # if x_ == 0:
        # continue
        position = [coord[0], coord[1]]
        # if y_ % 2 == 0 and x_ < 4:
        #     cv2.imshow(str(y_) + '-' + str(x_),
        #                images[position[1]:position[1] + cell_height, position[0]:position[0] + cell_width])
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()

        cv2.rectangle(images, (position[0], position[1]),
                      (position[0] + cell_width, position[1] + cell_height), (255, 255, 0),
                      thickness=2)

        temp.append(images[position[1]:position[1] + cell_height, position[0]:position[0] + cell_width])


        coord[0] += cell_width + margin[0]

    img_arr.append(temp)
    coord[1] += cell_height + margin[1]
    coord[0] = x

img_arr = np.array(img_arr)
img_arr.shape  # return (row, col, height, width)

(20, 8, 50, 153, 3)

In [ ]:
for y in img_arr:
    for x in y:

        img_indiv, ct, hi = cv2.findContours(x, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        rect = [cv2.boundingRect(each) for each in ct]
        rect = sorted(rect)

        for rec in rect:
            cv2.circle(img_indiv, (rec[0], rec[1]), 10, (0,0,255), -1)
            cv2.circle(img_indiv, (rec[0]+rec[2], rec[1]+ rec[3]), 10, (0,0,255), -1)
            cv2.rectangle(img_indiv, (rec[0], rec[1]), (rec[0]+rec[2], rec[1]+rec[3]), (0,255,0),3)
        cv2.imshow('a', img_indiv)
        cv2.waitKey(0)
        cv2.destroyAllWindows()



In [ ]:
resize = cv2.resize(images, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
cv2.imshow("area", resize)
cv2.waitKey(0)
cv2.destroyAllWindows()